# COVID-19 and Air Quality Index(AQI)
### This is the ETL process for analysis the relationship between COVID-19 and AQI, which included the bed status in each hospital by counties. 

In [1]:
# import dependences
import pandas as pd
from sqlalchemy import create_engine

### Loading data from csv files

In [22]:
confirmed_cases = pd.read_csv("Resources/US_comfirmed_cases.csv")
confirmed_cases.head()

,last_update,state,county_name,county_name_long,fips_code,lat,lon,NCHS_urbanization,total_population,confirmed,confirmed_per_100000,deaths,deaths_per_100000
0,2020-05-23 00:32:41,Alabama,Autauga,"Autauga, Alabama, US",1001.0,32.539527,-86.644082,Medium metro,55200.0,149,269.93,3,5.43
1,2020-05-23 00:32:41,Alabama,Baldwin,"Baldwin, Alabama, US",1003.0,30.727750,-87.722071,Small metro,208107.0,271,130.22,9,4.32
2,2020-05-23 00:32:41,Alabama,Barbour,"Barbour, Alabama, US",1005.0,31.868263,-85.387129,Non-core,25782.0,104,403.38,1,3.88
3,2020-05-23 00:32:41,Alabama,Bibb,"Bibb, Alabama, US",1007.0,32.996421,-87.125115,Large fringe metro,22527.0,55,244.15,1,4.44
4,2020-05-23 00:32:41,Alabama,Blount,"Blount, Alabama, US",1009.0,33.982109,-86.567906,Large fringe metro,57645.0,49,85.00,1,1.73


In [3]:
hospital_bed = pd.read_csv("Resources/hospital_bed.csv")
hospital_bed.head()

,X,Y,OBJECTID,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_ADDRESS,HQ_ADDRESS1,HQ_CITY,HQ_STATE,HQ_ZIP_CODE,...,CNTY_FIPS,FIPS,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE
0,-81.510688,41.133034,3001,Western Reserve Hospital (FKA Summa Western Re...,Short Term Acute Care Hospital,1900 23rd St,NaN,Cuyahoga Falls,OH,44223.0,...,153.0,39153.0,272.0,83.0,12,12,0.0,0.430764,189,2.0
1,-81.500489,41.080189,3002,Summa Health System - Akron Campus,Short Term Acute Care Hospital,525 E Market St,NaN,Akron,OH,44304.0,...,153.0,39153.0,649.0,584.0,64,64,0.0,0.540098,65,30.0
2,-81.589456,41.014805,3003,Summa Health System - Barberton Campus,Short Term Acute Care Hospital,155 5th St Ne,NaN,Barberton,OH,44203.0,...,153.0,39153.0,271.0,158.0,12,12,0.0,0.628819,113,0.0
3,-81.532470,41.078325,3004,Akron General,Short Term Acute Care Hospital,1 Akron General Ave,NaN,Akron,OH,44307.0,...,153.0,39153.0,383.0,383.0,34,34,0.0,0.706661,0,10.0
4,-81.513552,41.096977,3005,Crystal Clinic Orthopaedic Center,Short Term Acute Care Hospital,444 N Main St,NaN,Akron,OH,44310.0,...,153.0,39153.0,94.0,59.0,10,10,0.0,0.431066,35,2.0


In [4]:
aqi = pd.read_csv("Resources/annual_aqi_by_county_2020.csv")
aqi.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,Alabama,Baldwin,2020,11,11,0,0,0,0,0,48,39,20,0,0,0,0,11,0
1,Alabama,Clay,2020,5,5,0,0,0,0,0,31,31,15,0,0,0,0,5,0
2,Alabama,DeKalb,2020,59,59,0,0,0,0,0,45,40,32,0,0,58,0,1,0
3,Alabama,Etowah,2020,8,8,0,0,0,0,0,40,40,28,0,0,0,0,8,0
4,Alabama,Jefferson,2020,32,26,6,0,0,0,0,63,54,35,1,5,9,0,15,2


### Grouping up the total bed by counties with fips-codes, and extract three main columns to use in the project

In [19]:
# use groupby fips to find the total number of beds and potential increae beds in one county
total_bed = hospital_bed.groupby(['FIPS']).sum()['NUM_LICENSED_BEDS']
potential_bed = hospital_bed.groupby(['FIPS']).sum()['Potential_Increase_In_Bed_Capac']
# make the above values into one data frame
bed_df = pd.DataFrame({"total_bed":total_bed,
                      "potential_inc_bed":potential_bed})
# reset the data frame index
bed_df = bed_df.reset_index()
# rename the column name
bed_df = bed_df.rename(columns={"FIPS":"fips"})
# set new index
bed_df.set_index('fips',inplace=True)
# view the first five rows of the data
bed_df.head()

,total_bed,potential_inc_bed
fips,,
1001.0,85.0,30
1003.0,386.0,24
1005.0,74.0,44
1007.0,35.0,10
1009.0,25.0,0


### Extract the data needs from the covid-19 confirmed and deaths columns

In [35]:
# copy the columns we want into new dataset
confirmed_cols = ["state", "county_name", "fips_code","total_population","confirmed","deaths"]
confirmed_df = confirmed_cases[confirmed_cols].copy()
# rename the columns for better understanding
confirmed_df = confirmed_df.rename(columns={"county_name":"county",
                                            "fips_code":"fips"})
# create unique id for the data frame
confirmed_df.index = [x for x in range(1, len(confirmed_df.values)+1)]
confirmed_df.index.name = 'id'
# view the first five rows of the dataset
confirmed_df.head()

,state,county,fips,total_population,confirmed,deaths
id,,,,,,
1,Alabama,Autauga,1001.0,55200.0,149,3
2,Alabama,Baldwin,1003.0,208107.0,271,9
3,Alabama,Barbour,1005.0,25782.0,104,1
4,Alabama,Bibb,1007.0,22527.0,55,1
5,Alabama,Blount,1009.0,57645.0,49,1


### Extract the datas need for air quality analyze process

In [40]:
# copy the columns we wanted from the dataset
aqi_cols = ["State","County","Days with AQI","Good Days","Days PM2.5","Days PM10"]
aqi_df = aqi[aqi_cols].copy()
# rename the columns for easy access later
aqi_df = aqi_df.rename(columns={"State":"state",
                                "County":"county",
                                "Days with AQI":"aqi_days",
                                "Good Days":"good_days",
                                "Days PM2.5":"pm2p5",
                                "Days PM10":"pm10"})
# set the unique index value for aqi table
aqi_df.index = [x for x in range(1, len(aqi_df.values)+1)]
aqi_df.index.name = 'id'
# view the first five rows of the dataset
aqi_df.head()

,state,county,aqi_days,good_days,pm2p5,pm10
id,,,,,,
1,Alabama,Baldwin,11,11,11,0
2,Alabama,Clay,5,5,5,0
3,Alabama,DeKalb,59,59,1,0
4,Alabama,Etowah,8,8,8,0
5,Alabama,Jefferson,32,26,15,2


### Connect to local database

In [42]:
pw = "enter your password here"
conn_url = f"postgres:{pw}@localhost:5432/covid_aqi_db"
engine = create_engine(f'postgresql://{conn_url}')

In [43]:
# check for tables
engine.table_names()

['confirmed', 'bed', 'aqi']

### Use pandas to load csv converted DataFrame into database

In [ ]:
confirmed_df.to_sql(name='confirmed', con=engine, if_exists='append', index=True)

In [ ]:
bed_df.to_sql(name='bed', con=engine, if_exists='append', index=True)

In [48]:
aqi_df.to_sql(name='aqi', con=engine, if_exists='append', index=True)

### Confirm data has been added by querying the customer_location table

In [49]:
pd.read_sql_query('select * from confirmed', con=engine).head()

,id,state,county,fips,total_population,confirmed,deaths
0,1,Alabama,Autauga,1001.0,55200.0,149,3
1,2,Alabama,Baldwin,1003.0,208107.0,271,9
2,3,Alabama,Barbour,1005.0,25782.0,104,1
3,4,Alabama,Bibb,1007.0,22527.0,55,1
4,5,Alabama,Blount,1009.0,57645.0,49,1


In [50]:
pd.read_sql_query('select * from bed', con=engine).head()

,fips,total_bed,potential_inc_bed
0,1001,85,30
1,1003,386,24
2,1005,74,44
3,1007,35,10
4,1009,25,0


In [51]:
pd.read_sql_query('select * from aqi', con=engine).head()

,id,state,county,aqi_days,good_days,pm2p5,pm10
0,1,Alabama,Baldwin,11,11,11,0
1,2,Alabama,Clay,5,5,5,0
2,3,Alabama,DeKalb,59,59,1,0
3,4,Alabama,Etowah,8,8,8,0
4,5,Alabama,Jefferson,32,26,15,2
